# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [39]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time
import numpy as np

BATCH_SIZE = 128
NUM_EPOCHS = 100

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw


Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw


Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw


Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw
Processing...
Done!


Then, we define the model, object function and optimizer that we use to classify.

In [27]:
class SimpleNet(nn.Module):
# TODO:define model
# Lenet-5
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size = 5, padding = 2)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool1 = nn.AvgPool2d(kernel_size = 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size = 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool2 = nn.AvgPool2d(kernel_size = 2)
        self.conv3 = nn.Conv2d(16, 120, kernel_size = 5)
        self.bn3 = nn.BatchNorm2d(120)
        self.linear1 = nn.Linear(120, 84)
        self.linear2 = nn.Linear(84, 10)
        self.output = nn.Softmax(dim = 1)
        
    def forward(self, x):
        out = self.bn1(self.conv1(x))
        out = self.pool1(out)
        out = self.bn2(self.conv2(out))
        out = self.pool2(out)
        out = self.bn3(self.conv3(out))
        out = out.reshape(-1, 120)
        out = self.linear1(out)
        out = self.linear2(out) 
        out = self.output(out)
        return out
    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Next, we can start to train and evaluate!

In [40]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    model.train()
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        
        
    model.eval()
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    with torch.no_grad():
        Labels = torch.tensor([])
        predicts = torch.tensor([])
        for images, labels in tqdm(train_loader):
            outputs = model(images)
            predicts = torch.cat((predicts, outputs), 0)
            labels = labels.to(torch.float32)
            labels = labels.reshape(-1,1)
            Labels = torch.cat((Labels, labels), 0)
        
        
        predicts = predicts.cpu().numpy()
        Labels = Labels.detach().cpu().numpy()
        Pre = np.zeros_like(Labels)
        for i in range(predicts.shape[0]):
            Pre[i] = np.argmax(predicts[i])

        train_accuracy = np.mean(Pre == Labels)
        
        Labels = torch.tensor([])
        predicts = torch.tensor([])
        for images, labels in tqdm(test_loader):
            outputs = model(images)
            predicts = torch.cat((predicts, outputs), 0)
            labels = labels.to(torch.float32)
            labels = labels.reshape(-1,1)
            Labels = torch.cat((Labels, labels), 0)
            
        predicts = predicts.cpu().numpy()
        Labels = Labels.detach().cpu().numpy()
        Pre = np.zeros_like(Labels)
        for i in range(predicts.shape[0]):
            Pre[i] = np.argmax(predicts[i])
        
        test_accuracy = np.mean(Pre == Labels)
        print('Training accuracy: %0.2f%%' % (train_accuracy*100))
        print('Testing accuracy: %0.2f%%' % (test_accuracy*100))


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

Training accuracy: 88.49%
Testing accuracy: 89.44%


  0%|▏                                                                                 | 1/468 [00:00<00:57,  8.12it/s]

Training accuracy: 89.22%
Testing accuracy: 89.89%


  0%|▏                                                                                 | 1/468 [00:00<00:56,  8.32it/s]

...

Training accuracy: 93.01%
Testing accuracy: 92.76%


  0%|                                                                                          | 0/468 [00:00<?, ?it/s]

Training accuracy: 93.03%
Testing accuracy: 92.75%


100%|██████████████████████████████████████████████████████████████████████████████████| 78/78 [00:11<00:00,  6.74it/s]


Training accuracy: 93.08%
Testing accuracy: 92.74%


#### Q5:
Please print the training and testing accuracy.